In [ ]:
!wget https://raw.githubusercontent.com/mlzoo/multi-step-regression-tutorials/main/Raotbl6.csv

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

df = pd.read_csv('Raotbl6.csv')
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
df.drop('date', axis=1, inplace=True)

In [ ]:
target = 'rgnp'

X_train = df[: int(len(df) * 0.8)].iloc[:-1]
y_train = df[: int(len(df) * 0.8)].shift(-1).dropna()

X_test = df[int(len(df) * 0.8) :].iloc[:-1]
y_test = df[int(len(df) * 0.8) :].shift(-1).dropna()

model = MultiOutputRegressor(lgb.LGBMRegressor(objective='regression')).fit(X_train, y_train)

In [ ]:
results = []
data = X_test
for i in range(12):
    data = pd.DataFrame(model.predict(data), columns=data.columns, index=data.index)
    results.append(data)


In [ ]:
# make future targets 
for i in range(12):
    df['rgnp_{}'.format(i+1)] = df['rgnp'].shift(-i-1)

df.dropna(inplace=True)

targets = [item for item in df.columns if 'rgnp_' in item]

X_train = df.drop(targets, axis=1)[: int(len(df) * 0.8)]
y_train = df[targets][: int(len(df) * 0.8)]

X_test = df.drop(targets, axis=1)[int(len(df) * 0.8) :]
y_test = df[targets][int(len(df) * 0.8) :]

In [ ]:
models = []
for i in range(1, 13):
    
    model = lgb.LGBMRegressor(objective='regression').fit(X_train, y_train['rgnp_{}'.format(i)])
    X_train['rgnp_{}'.format(i)] = y_train['rgnp_{}'.format(i)]
    models.append(model)

In [ ]:
final_results = []

for i in range(12):
    final_results.append(models[i].predict(X_test))
    X_test['rgnp_{}'.format(i + 1)] = results[i]['rgnp']
    
pred = pd.DataFrame(np.array(final_results).T, columns=y_test.columns, index=y_test.index)
(pred - y_test).abs().mean()